## Testing gpu-acelerated curbd on our data

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../")

import pyaldata as pyal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from tools.reports.report_initial import run_initial_report
from tools.params import Params, colors
from tools.dsp.preprocessing import preprocess
import tools.viz.mean_firing as firing
import tools.viz.dimensionality as dim
import tools.viz.utilityTools as vizutils
import tools.decoding.rrr as rrr
import tools.decoding.decodeTools as decutils
import tools.dataTools as dt
from tools.curbd import curbd_gpu_v2
import cupy as cp

# 20th of March

In [2]:
# Files 
session = 'M062_2025_03_20_14_00'
data_dir = f"/data/bnd-data/raw/M062/{session}"

areas=["MOp", "SSp", "CP", "VAL"]
df = pyal.load_pyaldata(data_dir)

field values_before_camera_trigger could not be converted to int.
field idx_before_camera_trigger could not be converted to int.
array field VAL_KSLabel could not be converted to int.
array field SSp_KSLabel could not be converted to int.
array field all_KSLabel could not be converted to int.
array field MOp_KSLabel could not be converted to int.
array field CP_KSLabel could not be converted to int.
field values_Sol_direction could not be converted to int.
field idx_Sol_direction could not be converted to int.
field values_Sol_duration could not be converted to int.
field idx_Sol_duration could not be converted to int.
field idx_sol_on could not be converted to int.
array field VAL_KSLabel could not be converted to int.
array field SSp_KSLabel could not be converted to int.
array field all_KSLabel could not be converted to int.
array field MOp_KSLabel could not be converted to int.
array field CP_KSLabel could not be converted to int.
array field VAL_KSLabel could not be converted to i

In [3]:
df_ = preprocess(df, only_trials=False, repair_time_varying_fields=['MotSen1_X', 'MotSen1_Y'])

Repairing columns ['MotSen1_X', 'MotSen1_Y']
Extending index to 47989 in trial: free and id: 711, inserting NaN.
Extending index to 47990 in trial: free and id: 711, inserting NaN.
Extending index to 47991 in trial: free and id: 711, inserting NaN.
Extending index to 47992 in trial: free and id: 711, inserting NaN.
Extending index to 47993 in trial: free and id: 711, inserting NaN.
Extending index to 47994 in trial: free and id: 711, inserting NaN.
Extending index to 47995 in trial: free and id: 711, inserting NaN.
Extending index to 47996 in trial: free and id: 711, inserting NaN.
Extending index to 47997 in trial: free and id: 711, inserting NaN.
Extending index to 47998 in trial: free and id: 711, inserting NaN.
Extending index to 47999 in trial: free and id: 711, inserting NaN.
Extending index to 47989 in trial: free and id: 711, inserting NaN.
Extending index to 47990 in trial: free and id: 711, inserting NaN.
Extending index to 47991 in trial: free and id: 711, inserting NaN.
Ext

/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assum

Combined every 3 bins
Resulting VAL_spikes ephys data shape is (NxT): (155, 16000)
Resulting SSp_spikes ephys data shape is (NxT): (118, 16000)
Resulting all_spikes ephys data shape is (NxT): (34, 16000)
Resulting MOp_spikes ephys data shape is (NxT): (163, 16000)
Resulting CP_spikes ephys data shape is (NxT): (356, 16000)


### Get trials

In [15]:
activity_ = []
for sol_dir in range(12):
    df_trials = pyal.select_trials(df_, df_.values_Sol_direction == 1)
    df_trials = df_trials.iloc[:-1] 
    trial_length = df_trials.MOp_rates.values[0].shape[0]

    activity = []
    neurons = []
    for area in areas:
        neurons.append(df_trials[f"{area}_rates"][0].shape[-1])
        activity.append(pyal.concat_trials(df_trials, f"{area}_rates").T)

    # Activity
    activity = np.concatenate(activity)
    activity_.append(activity[np.newaxis, :, :])

activity_ = np.concatenate(activity_)

# Regions
regions = []

start_idx = 0
for area, size in zip(areas, neurons):
    regions.append([area, np.arange(start_idx, start_idx + size)])
    start_idx += size  # Update the starting index for the next region

activity_.shape

(12, 792, 6400)

In [12]:
# Reshape
activity = activity.reshape(activity.shape[0], activity.shape[1] // trial_length, trial_length)  # Shape (N, M, tr)
activity = activity.transpose(1, 0, 2)  # Shape (M, N, T)
activity = activity[:50, :, :]

activity.shape

(32, 792, 200)

### Train gpu curbd

In [10]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()


mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

a_cpu = np.ndarray(100, dtype=np.float32)

a = cp.array(a_cpu)
print(a.nbytes)                          # 400
print(mempool.used_bytes())              # 512
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

0
0
0
400
512
512
0


In [12]:
# When the array goes out of scope, the allocated device memory is released
# and kept in the pool for future reuse.
a = None  # (or `del a`)
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1


0
512
0


In [21]:
# You can clear the memory pool by calling `free_all_blocks`.
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

20275200
20275200
0


In [39]:
activity_.shape

(12, 792, 6400)

In [49]:
# %%time

gcurbd = curbd_gpu_v2.gCURBD(
    dt_data=df_.bin_size[0],
    dt_factor=3,
    regions=regions,
    train_epochs=1
)

gcurbd.fit(activity_)


trial=0 pVar=-4.253386 chi2=30.401431


In [50]:
del gcurbd

In [51]:
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

3178173952
8879664128
0


In [52]:
# You can clear the memory pool by calling `free_all_blocks`.
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

3178173952
4500677120
0


In [38]:

from tools.curbd import curbd


model = curbd.trainMultiRegionRNN(np.squeeze(activity_[:1, :, :]),
                                  dtData=df_.bin_size[0],
                                  dtFactor=5,
                                  regions=regions,
                                  nRunTrain=5,
                                  verbose=True,
                                  nRunFree=5,
                                  plotStatus=False)


# [curbd_arr, curbd_labels] = curbd.computeCURBD(model)

trial=0 pVar=-85.629713 chi2=6370.406681
trial=1 pVar=-87.474960 chi2=6506.087903
trial=2 pVar=-87.477044 chi2=6506.241147


KeyboardInterrupt: 